##### AJUSTA T1 PRA TAMANHO DO FLAIR

In [ ]:
import nibabel as nib
import numpy as np
from scipy.ndimage import zoom

flair_path = 'Displasia/sub-00003/anat/sub-00003_acq-T2sel_FLAIR.nii.gz'
t1_path = 'Displasia/sub-00003/anat/sub-00003_acq-iso08_T1w.nii.gz' 
output_path = 't1_reduzido.nii.gz' # Caminho onde quer salvar

# Paths das imagens
flair_img = nib.load(flair_path)
t1_img = nib.load(t1_path)

# Data das imagens
flair_data = flair_img.get_fdata()
t1_data = t1_img.get_fdata()

# Shape das imagens
flair_shape = flair_data.shape
t1_shape = t1_data.shape

# Calcular o fator de escala para redimensionar a imagem T1 para o tamanho da imagem FLAIR
scaling_factor = np.array(flair_shape) / np.array(t1_shape)

# Redimensionar a imagem T1 para as dimensões da imagem FLAIR usando interpolação trilinear
t1_rescaled = zoom(t1_data, scaling_factor, order=1)  # order=1 para interpolação bilinear

# Salvar a imagem T1 escalada
t1_rescaled_img = nib.Nifti1Image(t1_rescaled, t1_img.affine)
nib.save(t1_rescaled_img, output_path) # Implementar lógica para salvar a imagem redimensionada e já usá-la na próxima célula

##### PRÉ-PROCESSAR MANTENDO FORMATO


In [ ]:
#!/usr/bin/env python3
import os
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime
from functools import partial
from concurrent.futures import as_completed
import os
import numpy as np
import ants
import logging
from antspynet.utilities import brain_extraction
import gc

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Desativa GPUs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # Supressão de logs detalhados do TensorFlow

MODALITY = 'flair' #BOTE 'flair' ou 't1'

# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# FUNÇÕES
# Winsorize -> reduz outliers, limitando os percentis inf e sup
def winsorize_image(image_data, lower_percentile=1, upper_percentile=99):
    lower_bound = np.percentile(image_data, lower_percentile)
    upper_bound = np.percentile(image_data, upper_percentile)
    winsorized_data = np.clip(image_data, lower_bound, upper_bound)
    return winsorized_data

# Normalization -> valores de voxels entre 0 e 1
def normalize_image(image_data):
    min_val = np.min(image_data)
    max_val = np.max(image_data)
    normalized_data = (image_data - min_val) / (max_val - min_val)
    return normalized_data

# Função para processar uma única imagem
def process_image(img_path, output_dir):
    output_path = os.path.join(output_dir, os.path.basename(img_path))
    try:
        logger.info(f"Inicio processamento: {img_path}")
        # Carrega a imagem
        image = ants.image_read(img_path)
        data = image.numpy()

        # Cria template pra máscara
        prob_mask = brain_extraction(image, modality=MODALITY) # MODALIDADE: 'flair' ou 't1'
        logger.info(f"Template obtido.")

        # Cria a máscara
        mask = ants.get_mask(prob_mask, low_thresh=0.5)
        logger.info(f"Máscara aplicada.")

        # Máscara do cérebro e extração
        brain_masked = ants.mask_image(image, mask)
        logger.info(f"Extração.")
        
        # Winsorizing
        data = winsorize_image(data)
        logger.info(f"Winsorized.")

        # Bias Field Correction
        image = ants.from_numpy(data, origin=image.origin, spacing=image.spacing, direction=image.direction)
        image = ants.n4_bias_field_correction(image, shrink_factor=2)
        logger.info(f"Bias Corrigido.")

        # Normalização
        normalized_data = normalize_image(brain_masked.numpy())
        normalized_image = ants.from_numpy(normalized_data, origin=brain_masked.origin, spacing=brain_masked.spacing, direction=brain_masked.direction)

        logger.info(f"Imagem {img_path} processada.")

        ants.image_write(normalized_image, output_path)
        logger.info(f"Imagem salva: {os.path.basename(output_path)}")

        gc.collect()

        return normalized_image
        
    except Exception as e:
        logger.error(f"Erro ao processar a imagem {img_path}: {e}")
        return None

# DIRETÓRIOS
DIR_BASE = "/mnt/c/Users/Team Taiane/Desktop/ADNI/Dados publicos FLAIR que tem mask/Dados publicos FLAIR que tem mask"
DIR_OUTPUT = "/mnt/c/Users/Team Taiane/Desktop/ADNI/Dados_FLAIR_processados/FLAIR_processado_mascara_individual"
os.makedirs(DIR_OUTPUT, exist_ok=True)

# Checa o diretório de saída pra ver se alguma imagem já foi processada
already_processed = [file for file in os.listdir(DIR_OUTPUT)] 
# Lista de caminhos para as imagens brutas
image_paths = [os.path.join(DIR_BASE, file) for file in os.listdir(DIR_BASE) if file not in already_processed]

# Início do processamento
if __name__ == "__main__":
    start_time = datetime.now()
    logger.info(f"Início do processamento em: {start_time}")

    print(f"\n\nIMAGENS PROCESSADAS: {len(already_processed)}\nIMAGENS A PROCESSAR: {len(image_paths)}\n\n")

    # Função parcial para passar parâmetros fixos
    process_func = partial(process_image, output_dir=DIR_OUTPUT)

    # Processamento e salvamento de cada imagem usando ProcessPoolExecutor
    with ProcessPoolExecutor(max_workers=6) as executor: #max_workers define o número máximo de processos paralelos

        # dependendo do pc, é melhor fazer um proceso só, pois paralelizar pode deixar cada processo mais demorado sem hardware que aguente

        futures = [executor.submit(process_func, img_path) for img_path in image_paths]
        
        for future in as_completed(futures):
            future.result()  # Pega o resultado para garantir que exceções sejam lançadas

    # Fim do processamento
    end_time = datetime.now()
    logger.info(f"Término do processamento em: {end_time}")
    logger.info(f"Duração total: {end_time - start_time}")


##### VISUALIZAÇÃO DE IMAGENS

In [ ]:
import matplotlib.pyplot as plt
from nilearn.image import load_img
import os

img_path = "C:/Users/Team Taiane/Desktop/ADNI/FULL_ADNI/NIFTI_PROCESSED/test/cn/I495042.nii.gz"

# Definir as coordenadas de corte
x_slices = [60, 90, 120]  # Coordenadas para as fatias no eixo x
y_slices = [60, 90, 120]  # Coordenadas para as fatias no eixo y
z_slices = [60, 90, 120]  # Coordenadas para as fatias no eixo z

# Carregar a imagem do cérebro e a máscara de lesão
img_brain = load_img(img_path)

# Obter os dados das imagens como arrays numpy
brain_data = img_brain.get_fdata()

print(f"{os.path.basename(img_path)} - shape: {brain_data.shape}")

# Criar a figura para plotagem
fig, axes = plt.subplots(3, 3, figsize=(20, 10))  # Plotando 3 linhas e 6 colunas para múltiplas fatias

# Plotar as fatias para o eixo x
for i, x in enumerate(x_slices):
    brain_sagittal = brain_data[x, :, :]  # Fatia sagital no eixo x
    axes[0, i].imshow(brain_sagittal.T, cmap="gray", origin="lower")  # Exibe a fatia do cérebro
    axes[0, i].set_title(f"Fatia Sagital x={x}")
    axes[0, i].axis("off")  # Desativar os eixos

# Plotar as fatias para o eixo y
for i, y in enumerate(y_slices):
    brain_coronal = brain_data[:, y, :]  # Fatia coronal no eixo y
    axes[1, i].imshow(brain_coronal.T, cmap="gray", origin="lower")  # Exibe a fatia do cérebro
    axes[1, i].set_title(f"Fatia Coronal y={y}")
    axes[1, i].axis("off")  # Desativar os eixos

# Plotar as fatias para o eixo z
for i, z in enumerate(z_slices):
    brain_axial = brain_data[:, :, z]  # Fatia axial no eixo z
    axes[2, i].imshow(brain_axial.T, cmap="gray", origin="lower")  # Exibe a fatia do cérebro
    axes[2, i].set_title(f"Fatia Axial z={z}")
    axes[2, i].axis("off")  # Desativar os eixos

# Ajustar o layout para garantir que os títulos não se sobreponham
plt.tight_layout()
plt.show()